# [Module 4.3] Personalize 캠페인과 실시간 상호 작용 하기

이 노트북은 사용자의 실시간 행동에 반응하는 기능을 추가하는 과정을 안내합니다. 영화를 탐색하는 동안 사용자의 의도가 변경되면, 해당 동작에 따라 수정된 추천 영화 목록들이 표시됩니다.
이 노트북은 다음과 같은 작업을 합니다.
- 이벤트 추적기 생성
- 이전 노트북에서 실행한 처음 추천 결과 보기
- 새로운 영화를 클릭했다고 가정하고, 이벤트 트랙커 업데이트 후에 두 번째 추천 결과 보기
- 다른 새로운 영화를 클릭했다고 가정하고, 이벤트 트랙커 업데이트 후에 세 번째 추천 결과 보기

In [1]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time
import uuid
from random import randint

아래 코드 셀은 이전 notebook에서 저장했던 공유 변수들을 불러옵니다.

In [2]:
%store -r


생성할 오브젝트의 끝에 임의의 숫자를 부여하기 위해 suffix 정의

In [3]:
# suffix = str(np.random.uniform())[4:9]

In [4]:
# Setup and Config
# Recommendations from Event data
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

# Establish a connection to Personalize's Event Streaming
personalize_events = boto3.client(service_name='personalize-events')

## 이벤트 추적기 생성

추천 시스템이 실시간 이벤트에 응답하기 전에 이벤트 추적기(Event Tracker)가 필요합니다. 아래 코드 셀에서 이벤트 추적기 하나를 생성하고 이 실습에서 계속 사용하겠습니다. 이벤트 추적기 이름을 `MovieClickTracker`로 부여하였지만, 여러분들이 자유롭게 지정하실 수 있습니다.<br>
위의 추적 ID가 표시되며, 이 ID는 변수에 할당되었으므로 추가 조치가 필요하지 않습니다. 아래 코드 셀을 실행하면 나중에 추천 영화 목록을 출력할 수 있도록 합니다.

In [5]:
import time


response = personalize.create_event_tracker(
    name='MovieClickTracker',
    datasetGroupArn=dataset_group_arn
)

# wait tracker creation
time.sleep(30)

print(response['eventTrackerArn'])
print(response['trackingId'])
TRACKING_ID = response['trackingId'] 

arn:aws:personalize:ap-northeast-2:057716757052:event-tracker/ec2d140e
be24b680-a800-4087-98f4-e6c3bcaa6f3b


In [6]:
event_tracker_arn = response['eventTrackerArn']

#### help 함수(영화 타이틀, 장르 리턴)



In [7]:
# First load items into memory
items_all = pd.read_csv('./ml-1m/movies.dat',sep='::', encoding='latin1',names=['ITEM_ID', 'TITLE', 'GENRE'],)
items=items_all.copy()
items['to_keep'] = items['ITEM_ID'].apply(lambda x:x in unique_items)
items=items[items['to_keep']]
del items['to_keep']
print(items.tail())

def get_movie_title(movie_id):
    """
    Takes in an ID, returns a title
    """
    movie_id = int(movie_id)
    movie_title=items[items['ITEM_ID']==movie_id]['TITLE']
    return (movie_title.tolist())

def get_movie_genre(movie_id):
    """
    Takes in an ID, returns a genre
    """
    movie_id = int(movie_id)
    movie_genre=items[items['ITEM_ID']==movie_id]['GENRE']
    return (movie_genre.tolist())



      ITEM_ID                       TITLE           GENRE
3878     3948     Meet the Parents (2000)          Comedy
3879     3949  Requiem for a Dream (2000)           Drama
3880     3950            Tigerland (2000)           Drama
3881     3951     Two Family House (2000)           Drama
3882     3952       Contender, The (2000)  Drama|Thriller


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  from ipykernel import kernelapp as app


## 처음 추천 결과 확인

이전 노트북에서 아래 유저에 대해서 (예: 4375)  추천 리스트를 확인 합니다. <br>

In [8]:
print("user_id: ", user_id)

user_id:  4375


In [9]:
recommendations_df

,OriginalRecs,Genre
0,"[Big Sleep, The (1946)]",[Film-Noir|Mystery]
1,[Key Largo (1948)],[Crime|Drama|Film-Noir|Thriller]
2,[Chinatown (1974)],[Film-Noir|Mystery|Thriller]
3,[Sunset Blvd. (a.k.a. Sunset Boulevard) (1950)],[Film-Noir]
4,"[Manchurian Candidate, The (1962)]",[Film-Noir|Thriller]
5,[Shadow of a Doubt (1943)],[Film-Noir|Thriller]
6,"[Murder, My Sweet (1944)]",[Film-Noir|Thriller]
7,[On the Waterfront (1954)],[Crime|Drama]
8,[Singin' in the Rain (1952)],[Musical|Romance]
9,"[Philadelphia Story, The (1940)]",[Comedy|Romance]


## 사용자 행동 시뮬레이션

아래 코드 셀은 특정 item과 상호 작용하는 사용자를 시뮬레이트하는 코드 샘플을 제공하며, 시작할 때와 다른 추천 목록을 얻습니다.

아래 ```send_movie_click(USER_ID, ITEM_ID)``` 는 다음과 같은 작업을 하여 실시간 이벤트를 반영 합니다.

- 유저에 대한 세션이 없으면 SessionID를 생성하고, 있으면 이 유저의 SessionID를 불러온다
- event를 정의하고 event에 item_id를 할당한다
- event를 json 형태로 바꾼다
- 위에서 생성한 event tranker의 put_event에 위 json파일을 인자로 넘겨서 호출한다.

#### Help 함수

In [10]:
session_dict = {}

def send_movie_click(USER_ID, ITEM_ID):
    """
    Simulates a click as an envent
    to send an event to Amazon Personalize's Event Tracker
    """
    # Configure Session
    try:
        session_ID = session_dict[USER_ID]
    except:
        session_dict[USER_ID] = str(uuid.uuid1())
        session_ID = session_dict[USER_ID]
   
    value=randint(0,5)
    
    # Configure Properties:
    event = {
    "itemId": str(ITEM_ID),
    "eventValue": value
    }
    event_json = json.dumps(event)
    
    # Make Call
    personalize_events.put_events(
    trackingId = TRACKING_ID, # 이벤트트래커에서 생성한 아이디
    userId= USER_ID,
    sessionId = session_ID,
    eventList = [{
        'sentAt': int(time.time()),
        'eventType': 'RATING',
        'properties': event_json
        }]
)
    
# Help 함수

def rec_item_list(item_list, clicked_title, clicked_genre):
    recommendation_title_list = []
    recommendation_genre_list = []

    for item in item_list:
        title = get_movie_title(item['itemId'])
        genre = get_movie_genre(item['itemId'])
        recommendation_title_list.append(title)
        recommendation_genre_list.append(genre)    

    title_df = pd.DataFrame(recommendation_title_list, columns=[clicked_title])
    genre_df = pd.DataFrame(recommendation_genre_list, columns=[clicked_genre])        
    recommendations_df = title_df.join(genre_df)

    return recommendations_df                                                    

def get_new_recommend(user_id,campaign_arn, recommendations_df, title, genre):    
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = campaign_arn,
        userId = str(user_id),
    )

    item_list = get_recommendations_response['itemList']
    new_recs_df = rec_item_list(item_list,title,  genre)
    new_recommendations_df = pd.concat([recommendations_df,new_recs_df ], axis=1)

    print("Recommendations for user: ", user_id)
    return new_recommendations_df
    

## Movie 선택 및 두번째 추천 리스트

이번에는 토이 스토리를 클릭했다고 가정하겠습니다.<br>
이후에 추천의 변화를 보시죠.<br>
"movie_title: Toy Story (1995), movie_genre: Animation|Children's|Comedy" 선택에 따른 새로운 추천의 변화
- 아래 테이블의 결과를 보는 방법은 첫번째, 두번째 컬럼은 오지널 추천 영화의 제목 및 장르 입니다. <br>
세번째의 컬럼의 맨위는 "클릭한 영화 제목" 이고 이하는 추천 리스트 입니다. 그리고 네번째 컬럼의 맨위는 세번째 컬럼의 각 해당하는 영화의 장르 입니다.

**새로운 추천이 시간에 대한 입력값이 들어가기에 추천 결과가 실행시마다 상이할 수 있습니다.**


In [12]:
%store -r
pd.options.display.max_columns = 10


# Pick a movie, we will use ID 100 or (Postino, Il (The Postman) (1994),)
##########################
# 영화 선택
##########################
movie_to_click = 1 # movie_title: Toy Story (1995), movie_genre: Animation|Children's|Comedy
##########################
movie_title_clicked = get_movie_title(movie_to_click)
movie_genre_clicked = get_movie_genre(movie_to_click)
print("user_id: {}, movie_id_clicked: {}, movie_title: {}, movie_genre: {}".format(user_id, movie_to_click, movie_title_clicked[0], movie_genre_clicked[0]))
# 이벤트 추적기 put_event 호출
send_movie_click(USER_ID=str(user_id), ITEM_ID=movie_to_click)
# 새로운 추천 결과
recommendations_df = get_new_recommend(user_id,user_personalization_campaign_arn, recommendations_df,movie_title_clicked, movie_genre_clicked )    
recommendations_df


user_id: 4375, movie_id_clicked: 1, movie_title: Toy Story (1995), movie_genre: Animation|Children's|Comedy
Recommendations for user:  4375


,OriginalRecs,Genre,"(Toy Story (1995),)","(Animation|Children's|Comedy,)"
0,"[Big Sleep, The (1946)]",[Film-Noir|Mystery],Child's Play 2 (1990),Horror
1,[Key Largo (1948)],[Crime|Drama|Film-Noir|Thriller],"Nightmare on Elm Street 3: Dream Warriors, A (...",Horror
2,[Chinatown (1974)],[Film-Noir|Mystery|Thriller],Pet Sematary (1989),Horror
3,[Sunset Blvd. (a.k.a. Sunset Boulevard) (1950)],[Film-Noir],Child's Play (1988),Horror
4,"[Manchurian Candidate, The (1962)]",[Film-Noir|Thriller],Friday the 13th (1980),Horror
5,[Shadow of a Doubt (1943)],[Film-Noir|Thriller],I Still Know What You Did Last Summer (1998),Horror|Mystery|Thriller
6,"[Murder, My Sweet (1944)]",[Film-Noir|Thriller],Scream 2 (1997),Horror|Thriller
7,[On the Waterfront (1954)],[Crime|Drama],"Nightmare on Elm Street 5: The Dream Child, A ...",Horror
8,[Singin' in the Rain (1952)],[Musical|Romance],Scream 3 (2000),Horror|Mystery|Thriller
9,"[Philadelphia Story, The (1940)]",[Comedy|Romance],Tales From the Crypt Presents: Demon Knight (1...,Horror


## 또 다른 Movie 선택 및 세번째 추천 리스트

그 다음에는 알라딘을 클릭한 것을 시뮬레이션 합니다.
"movie_title: Aladdin (1992), movie_genre: Animation|Children's|Comedy|Musical" 선택에 따른 새로운 추천의 변화
- 아래 테이블의 결과를 보는 방법은 첫번째, 두번째, 세번째, 네번째는 위에서 확인한 것과 동일 합니다.<br>
다섯번째의 컬럼의 맨위는 이번에 "클릭한 영화 제목" 이고 이하는 추천 리스트 입니다. 그리고 여섯번째 컬럼의 맨위는 다섯번째 컬럼의 각 해당하는 영화의 장르 입니다.

**새로운 추천이 시간에 대한 입력값이 들어가기에 추천 결과가 실행시마다 상이할 수 있습니다.**


In [13]:
import time
time.sleep(30) # 이벤트 트랙커가 업데이트 되게 잠시 기다립니다.

In [14]:
# Pick a movie, we will use ID 100 or (Postino, Il (The Postman) (1994),)
##########################
# 영화 선택
##########################
movie_to_click = 588 # movie_title: Aladdin (1992), movie_genre: Animation|Children's|Comedy|Musical
##########################
movie_title_clicked = get_movie_title(movie_to_click)
movie_genre_clicked = get_movie_genre(movie_to_click)
print("user_id: {}, movie_id_clicked: {}, movie_title: {}, movie_genre: {}".format(user_id, movie_to_click, 
                                                                                   movie_title_clicked[0], movie_genre_clicked[0]))
# 이벤트 추적기 put_event 호출
send_movie_click(USER_ID=str(user_id), ITEM_ID=movie_to_click)
recommendations_df = get_new_recommend(user_id,user_personalization_campaign_arn, recommendations_df,
                                       movie_title_clicked, movie_genre_clicked )    
recommendations_df

user_id: 4375, movie_id_clicked: 588, movie_title: Aladdin (1992), movie_genre: Animation|Children's|Comedy|Musical
Recommendations for user:  4375


,OriginalRecs,Genre,"(Toy Story (1995),)","(Animation|Children's|Comedy,)","(Aladdin (1992),)","(Animation|Children's|Comedy|Musical,)"
0,"[Big Sleep, The (1946)]",[Film-Noir|Mystery],Child's Play 2 (1990),Horror,Pinocchio (1940),Animation|Children's
1,[Key Largo (1948)],[Crime|Drama|Film-Noir|Thriller],"Nightmare on Elm Street 3: Dream Warriors, A (...",Horror,Charlotte's Web (1973),Animation|Children's
2,[Chinatown (1974)],[Film-Noir|Mystery|Thriller],Pet Sematary (1989),Horror,Groundhog Day (1993),Comedy|Romance
3,[Sunset Blvd. (a.k.a. Sunset Boulevard) (1950)],[Film-Noir],Child's Play (1988),Horror,Toy Story 2 (1999),Animation|Children's|Comedy
4,"[Manchurian Candidate, The (1962)]",[Film-Noir|Thriller],Friday the 13th (1980),Horror,Fantasia (1940),Animation|Children's|Musical
5,[Shadow of a Doubt (1943)],[Film-Noir|Thriller],I Still Know What You Did Last Summer (1998),Horror|Mystery|Thriller,"Sixth Sense, The (1999)",Thriller
6,"[Murder, My Sweet (1944)]",[Film-Noir|Thriller],Scream 2 (1997),Horror|Thriller,"Shawshank Redemption, The (1994)",Drama
7,[On the Waterfront (1954)],[Crime|Drama],"Nightmare on Elm Street 5: The Dream Child, A ...",Horror,"Bug's Life, A (1998)",Animation|Children's|Comedy
8,[Singin' in the Rain (1952)],[Musical|Romance],Scream 3 (2000),Horror|Mystery|Thriller,"Lion King, The (1994)",Animation|Children's|Musical
9,"[Philadelphia Story, The (1940)]",[Comedy|Romance],Tales From the Crypt Presents: Demon Knight (1...,Horror,Raiders of the Lost Ark (1981),Action|Adventure


## 결론

사용자가 상호 작용하는 영화를 변경하여 추천 사항이 변경되었음을 알 수 있습니다. 이 시스템은 사용자가 item 모음과 상호 작용하는 모든 응용 프로그램으로 확장할 수 있습니다. 이러한 도구는 언제든지 사용 가능한 데이터로 가능한 것을 풀다운하여 탐색할 수 있습니다.

In [15]:
%store event_tracker_arn

Stored 'event_tracker_arn' (str)
